#### データソースを準備

In [8]:
from trafilatura import fetch_url, extract

url = 'https://yugioh-wiki.net/index.php?%A1%DA%A5%A2%A5%EB%A5%D0%A5%B9%A4%CE%CD%EE%B0%FD%A1%DB'
filename = 'deck_albaz.txt'

document = fetch_url(url)
text = extract(document)

if text is not None:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)
else:
    print("No text could be extracted from the document.")


#### チャンク分割

In [9]:
from langchain.document_loaders import TextLoader

loader = TextLoader(filename, encoding='utf-8')
raw_docs = loader.load()

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=100,
    chunk_overlap=0,
)

docs = text_splitter.split_documents(raw_docs)

print(len(docs))
print(docs[0])


Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 143, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 114, which is longer than the specified 100
Created a chunk of size 161, which is longer than the specified 100
Created a chunk of size 139, which is longer than the specified 100


97
page_content='【アルバスの落胤】 †\nデッキの概要 †\n自身と相手モンスターを素材にフィールド融合を行う《アルバスの落胤》を中心とした【正規融合】デッキ。' metadata={'source': 'deck_albaz.txt'}


#### embedding

In [10]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings

load_dotenv()

# OpenAIのEmbeddingモデルを読み込む
embeddings = OpenAIEmbeddings(deployment=os.environ["DEPLOYMENT_NAME_EMBEDDINGS"])

from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

#### 回答

In [22]:
from langchain.chains import  RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

retriever = db.as_retriever()

# チャットの組み立て
chat = AzureChatOpenAI(
    deployment_name=os.environ["DEPLOYMENT_NAME"],
    temperature=0
)

# RAGのためのchainを組み立て
rag_qa = RetrievalQA.from_chain_type(
    llm = chat,
    chain_type= "stuff",
    retriever = retriever,
    return_source_documents = True,
)

# 質問文のリストを作成
query_list = ["「アルバスの落胤」デッキのキーカードは？",
              "「アルバスの落胤」の攻は？",
              "「アルバスの落胤」デッキの弱点は？",
              "烙印融合のテキストを教えて",
              "「アルバスの落胤」デッキの派生デッキを3つ教えて",
              "「アルバスの落胤」デッキではリンクモンスターは何が採用される？",
              "「アルバスの落胤」デッキでよく使われるカウンター罠は？",
              "「アルバスの落胤」デッキの戦術は？",
              "「烙印融合」をサーチする方法を教えて",
              "「アルバスの落胤」デッキでは召喚権をどのモンスターに使う？",
              "「アルバスの落胤」デッキにティアラメンツを混ぜるメリットは？",
]

# LLMに渡すチャンクを格納する空リストを作成
context_list = []

# LLMからの回答を格納する空リストを作成
result_list = []

# rag_qaを回して、context_listとresult_listに選ばれたチャンクと回答を格納していく
for query in query_list:
    rag_result = rag_qa({"query": query})
    context_list.append(rag_result["source_documents"])
    result_list.append(rag_result["result"])


print("チャンクを表示")
print(*context_list, sep='\n')
print("*************")
print("回答を表示")
print(*result_list, sep='\n')


チャンクを表示
[Document(page_content='《アルバスの落胤》とのシナジーを最優先とし、他カテゴリ間のコンボに関係なく汎用的なカードを採用したデッキ。', metadata={'source': 'deck_albaz.txt'}), Document(page_content='【アルバスの落胤】 †\nデッキの概要 †\n自身と相手モンスターを素材にフィールド融合を行う《アルバスの落胤》を中心とした【正規融合】デッキ。', metadata={'source': 'deck_albaz.txt'}), Document(page_content='―《アルバスの落胤》のサポートカード\n―融合召喚を行うカード\n―相性の良いカード\n戦術 †', metadata={'source': 'deck_albaz.txt'}), Document(page_content='特に何も考えず《アルバスの落胤》に纏わるストーリー場面カードを投入していくだけで自然とこのタイプになる。\nこのデッキの弱点 †', metadata={'source': 'deck_albaz.txt'})]
[Document(page_content='共通効果の内、《アルバスの落胤》のリクルートが可能なものは後続の《アルバスの落胤》を展開して戦線維持を狙える。', metadata={'source': 'deck_albaz.txt'}), Document(page_content='《アルバスの落胤》とのシナジーを最優先とし、他カテゴリ間のコンボに関係なく汎用的なカードを採用したデッキ。', metadata={'source': 'deck_albaz.txt'}), Document(page_content='《アルバスの落胤》には、烙印を始めとする専用サポートや自身を名指し指定する融合モンスターが多く、それらを軸として構築していく。', metadata={'source': 'deck_albaz.txt'}), Document(page_content='―《アルバスの落胤》のサポートカード\n―融合召喚を行うカード\n―相性の良いカード\n戦術 †', metadata={'source': 'deck_albaz.txt'})]
[Docum